
# Task 1 – Daten-Preprocessing

Ziel: Rohdaten aus `data/` (insb. `buildings.csv`) explorieren, bereinigen, normalisieren und für weitere Schritte (Anreicherung / Visualisierung) vorbereiten.

## Pandas installation

In [2]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 11.0 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 11.0 MB/s  0:00:01 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd

buildings = pd.read_csv('../buildings.csv', dtype=str)
# Begründung: Viele Spalten enthalten gemischte / freie Werte 
# → zunächst Strings vermeiden fehlerhafte automatische Typkonvertierungen.
print(buildings.shape)
buildings.head()

(99, 715)


,ID,appellation,alternativeNames,addressCountry,addressState,addressLocality,addressZip,addressStreet,locationLng,locationLat,...,TEMPLATE_PROVIDERS_OF_1,TEMPLATE_PROVIDERS_OF_2,TEMPLATE_PROVIDERS_OF_3,TEMPLATE_PROVIDERS_OF_4,TEMPLATE_PROVIDERS_OF_5,TEMPLATE_PROVIDERS_OF_6,TEMPLATE_PROVIDERS_OF_7,TEMPLATE_PROVIDERS_OF_8,TEMPLATE_PROVIDERS_OF_9,TEMPLATE_PROVIDERS_OF_10
0,48554093-289c-492a-a060-735d4e8971e8,"Bad Buchau, Stiftskirche",NaN,Deutschland,Baden-Württemberg,Bad Buchau,88422,NaN,9.611163,48.0674772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,843e2427-35b0-41ef-9299-0b44fb61e495,"Bad Buchau, Abteigebäude",NaN,Deutschland,Baden-Württemberg,Bad Buchau,88422,Schloßplatz 2,9.610800099999999,48.06731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,301e0950-c804-11e9-8a8f-6315b3a88f02,"Leitheim, Schlossensemble, Weingärtnerhaus",NaN,Deutschland,Bayern,Leitheim,86687,Schloßstraße 5,10.883508912560208,48.7421674504304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,85c94cd0-c803-11e9-8a8f-6315b3a88f02,"Leitheim, Kirche St. Blasius",NaN,Deutschland,Bayern,Leitheim,86687,Schloßstraße 3,10.883714569872827,48.74199234101888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,f67ae610-c802-11e9-8a8f-6315b3a88f02,"Leitheim, Schloss",NaN,Deutschland,Bayern,Leitheim,86687,Schloßstraße 3,10.883350984238188,48.74199057228771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Erste Profilierung

1. Zähle Null/Leer-Werte pro Spalte (`''`, `' '`, `NaN`).
2. Ermittle Anteil gefüllter Werte für Kernfelder: `ID`, `appellation`, `verbaleDating`, `locationLat`, `locationLng`.
3. Erzeuge eine Kurztabelle (DataFrame) mit (Spaltenname, Non-Null %, Anzahl unterschiedlicher Werte, Beispielwerte).

In [13]:
raw = buildings.copy()
null_like = raw.replace({'': pd.NA, ' ': pd.NA})
profile = []
for col in null_like.columns:
    s = null_like[col]
    profile.append({
        'column': col,
        'non_null_pct': s.notna().mean()*100,
        'n_unique': s.nunique(dropna=True),
        'sample_values': ', '.join(s.dropna().unique()[:3])
    })
profile_df = pd.DataFrame(profile).sort_values('non_null_pct')
profile_df.head(15)

/tmp/ipykernel_138692/1338647739.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  null_like = raw.replace({'': pd.NA, ' ': pd.NA})


,column,non_null_pct,n_unique,sample_values
435,MARBLE_WORKERS_1,0.0,0,
444,MARBLE_WORKERS_10,0.0,0,
443,MARBLE_WORKERS_9,0.0,0,
442,MARBLE_WORKERS_8,0.0,0,
441,MARBLE_WORKERS_7,0.0,0,
440,MARBLE_WORKERS_6,0.0,0,
439,MARBLE_WORKERS_5,0.0,0,
438,MARBLE_WORKERS_4,0.0,0,
437,MARBLE_WORKERS_3,0.0,0,
436,MARBLE_WORKERS_2,0.0,0,


Fragen:

- Welche Spalten sind fast leer und sollten ggf. ausgelagert / ignoriert werden?
- Gibt es offensichtliche Duplikate bei `ID`?

## 2. Bereinigung `verbaleDating`

Spalte enthält verbale Zeitangaben / Bereiche wie: `"1000-2020, 1773-1776"`.

Aufgaben:

1. Zerlege `verbaleDating` in einzelne Segmente (Trennzeichen: Komma) → normalisiere Leerzeichen.
2. Identifiziere Bereiche (Pattern `YYYY-YYYY`) vs. Einzeljahre (`YYYY`).
3. Baue eine normalisierte Tabelle `building_dates`: (`building_id`, `date_raw`, `year_start`, `year_end`, `is_range`, `precision`).
4. Berechne je Gebäude: minimaler Start, maximaler Endwert → `chronology_min`, `chronology_max` und füge sie wieder dem Haupt-DataFrame hinzu.
5. Detektiere Ausreißer (z. B. Jahr < 1000 oder > aktuelles Jahr). Markiere sie für manuelle Prüfung.

In [14]:
import re
rows = []
for _, r in buildings[['ID','verbaleDating']].fillna('').iterrows():
    parts = [p.strip() for p in r.verbaleDating.split(',') if p.strip()]
    for p in parts:
        m_range = re.fullmatch(r'(\d{4})-(\d{4})', p)
        m_year  = re.fullmatch(r'(\d{4})', p)
        if m_range:
            y1, y2 = map(int, m_range.groups())
            rows.append((r.ID, p, y1, y2, True, 'year-range'))
        elif m_year:
            y = int(m_year.group(1))
            rows.append((r.ID, p, y, y, False, 'year'))
        else:
            # Nicht-parsbare Fälle separat behalten
            rows.append((r.ID, p, None, None, None, 'unparsed'))

building_dates = pd.DataFrame(rows, columns=['building_id','date_raw','year_start','year_end','is_range','precision'])


Optionale Normalisierung unparsed Tokens: RegEx erweitern (z. B. `ca. 1750`, `17. Jh.` → Mapping Tabellen). Dokumentiere Annahmen!

Metriken:

- Anteil parsebarer Segmente.
- Häufigste unparsed Muster (Top 10).

## 3. Geodaten-Validierung+Auswertung

1. Konvertiere `locationLat` / `locationLng` zu Float; markiere Zeilen, bei denen das misslingt.
2. Prüfe Wertebereiche (Lat ∈ [-90, 90], Lng ∈ [-180, 180]). Auf Hessen beschränken mit Geokoordinaten und Geoshape.....
3. Erstelle Spalten `coord_valid` (bool) und `coord_quality` (Enum: `valid`, `out_of_range`, `non_numeric`).
4. Optional: Entferne identische Koordinaten-Duplikate, falls mehrere Gebäude dieselben Koordinaten teilen sollen – dokumentiere Entscheidungslogik.

In [ ]:
def to_float(s):
    try:
        return float(s)
    except (TypeError, ValueError):
        return pd.NA
coords = buildings[['locationLat','locationLng']].applymap(to_float)
valid_lat = coords.locationLat.between(-90,90)
valid_lng = coords.locationLng.between(-180,180)
buildings['coord_valid'] = valid_lat & valid_lng

/tmp/ipykernel_138692/19503827.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  coords = buildings[['locationLat','locationLng']].applymap(to_float)


## 4. Normalisierung mehrfacher Rollen-/Personenfelder

Viele Spalten enden auf Sequenzen `_1`…`_10` (z. B. `ARCHITECTS_1`, `ARCHITECTS_2`, ...). Werteform: `uuid|Nachname, Vorname`.

Ziel: Long-Format-Relation `building_person_roles`:
(`building_id`, `role` (z. B. `ARCHITECTS`), `sequence` (Nummer), `person_id` (UUID), `person_label`).

Aufgaben:

1. Identifiziere alle Basis-Rollen (Teil vor letztem `_` + Ziffern).
2. Iteriere über alle diese Spalten, extrahiere Werte ≠ leer.
3. Teile an erster Pipe `|` → `person_id`, zweiter Teil `person_label` (Fallback: kompletter String falls kein `|`).
4. Entferne potenzielle Dubletten (gleiche Kombination building_id + role + person_id).
5. Erzeuge optionale Personentabelle `persons` (distinct `person_id`, `person_label`, `label_clean`).
6. Bereinige `person_label`: Whitespace trimmen, vereinheitliche Kommaspacing.

In [16]:
role_cols = [c for c in buildings.columns if re.search(r'_\d+$', c)]
base_roles = sorted(set(re.sub(r'_\d+$','', c) for c in role_cols))
rows = []
for role in base_roles:
    for i in range(1, 11):
        col = f'{role}_{i}'
        if col not in buildings.columns: 
            continue
        for _, r in buildings[['ID', col]].iterrows():
            val = r[col]
            if pd.isna(val) or not str(val).strip():
                continue
            if '|' in val:
                pid, label = val.split('|',1)
            else:
                pid, label = None, val
            rows.append((r.ID, role, i, pid, label.strip()))

building_person_roles = pd.DataFrame(rows, columns=['building_id','role','sequence','person_id','person_label'])
persons = (building_person_roles
           .dropna(subset=['person_label'])
           .groupby(['person_id','person_label'], dropna=False)
           .size().reset_index(name='count'))

Validierung:

- Wie viele Rollen wurden extrahiert?
- Top 3 Personen je Rolle.

## 5. Konsolidierung & Export

1. Schlanken Haupt-DataFrame erstellen (`buildings_core`): wesentliche Attribute (ID, appellation, address*, chronology_min/max, coord_valid, locationLat/Lng als Float).
2. Daten in Unterordner `processed/` exportieren:
   - `buildings_core.parquet`
   - `building_dates.parquet`
   - `building_person_roles.parquet`
   - `persons.parquet`
3. Zusätzlich CSV-Export für Interoperabilität (UTF-8, `index=False`).
4. README-Ergänzung (Kurzbeschreibung der erzeugten Dateien) – (Kann in Task 4 weiter genutzt werden).

In [17]:
out = Path('../processed'); out.mkdir(exist_ok=True)
buildings_core.to_parquet(out / 'buildings_core.parquet')
building_dates.to_parquet(out / 'building_dates.parquet')


NameError: name 'Path' is not defined

## 6. Qualitätsmetriken & Checks

Erzeuge kleine Metriken (als DataFrame oder Markdown):

- Parsebarkeit Datum (%).
- Anzahl unparsed Datumseinträge.
- Anzahl extrahierter Personenbeziehungen.
- #Distinct Personen.
- Anteil gültiger Koordinaten.

## OpenRefine

Ziel: Schnelle, reproduzierbare Bereinigung & Normalisierung zentraler Felder aus `buildings.csv` mittels OpenRefine.

> Fokus: Sicht auf typische OpenRefine-Operationen (Facets, Clustering, Transformations, Splits, Rekonsilierungsvorbereitung). Dauer: ca. 30–40 Minuten.

### 1. Import

- Lade `buildings.csv` in OpenRefine.
- Alle Spalten zunächst als Text lassen (kein Auto-Parsing von Zahlen/Datumsangaben aktivieren).
- Projektname: `buildings_raw`.

### 2. Grundlegende Sichtbarkeit

- Entferne (nur in der Ansicht, nicht dauerhaft) extrem leere Spalten via Facet → „Facet by blank“ und spätere Auswahl für Export.
- Erzeuge eine Text-Facet auf `appellation` → erkenne Varianten / Dubletten.

### 3. Bereinigung `verbaleDating`

Ziel: Segmentierung & Vor-Normalisierung für spätere Python-Verarbeitung.

Schritte:

1. Expression (GREL) Trim: `value.trim()` (Spalte bearbeiten → Zellen transformieren).
2. Ersetze mehrere Leerzeichen: `value.replace(/\s+/,' ')`.
3. Split bei Komma (Spaltenmenü → „Edit cells → Split multi-valued cells“ → Separator `,`).
4. Neue Spalte aus dieser (JSON-Serialisierung einzelner Werte für Kontrolle): `value` (Beibehalten). Optional: Werte mit Regex-Facet `^[0-9]{3,4}(-[0-9]{3,4})?$` filtern (parsbar vs. unparsed).
5. Erzeuge Booleanspalte `is_range`: GREL: `value.match(/^[0-9]{3,4}-[0-9]{3,4}$/) != null`.
6. Erzeuge Spalten `year_start` und `year_end`:
   - Falls Range: `value.match(/^(\d{3,4})-(\d{3,4})$/)[0]` & `[1]`
   - Falls Einzeljahr: `value` in beide kopieren.
7. Filtere Ausreißer: Facet `year_start` → Numeric Facet → Werte außerhalb 800–(aktuelles Jahr) markieren.

Hinweis: Exportiere Zwischenergebnis als `building_dates_openrefine.csv` für Abgleich mit Pandas-Parsing.

### 4. Personen-/Rollenfelder (Beispiel: ARCHITECTS)

Ziel: Long-Format Grundlage.

Schritte:

1. Wähle Spalten `ARCHITECTS_1` … `ARCHITECTS_10`.
2. „Edit columns → Join columns“ (Separator `||`), erzeuge Sammelspalte `ARCHITECTS_JOIN`.
3. Split multi-valued cells an `||` → leere entfernen.
4. Entferne Duplikate (Facet by text, Auswahl blank → ausschließen).
5. Extrahiere UUID & Label:
   - Neue Spalte `person_id`: GREL: `if(value.contains('|'), value.split('|')[0], null)`
   - Neue Spalte `person_label_raw`: GREL: `if(value.contains('|'), value.split('|')[1], value)`
6. Clean Label: Neue Spalte `person_label`: `person_label_raw.trim().replace(/\s*,\s*/, ', ')`.
7. Cluster (Edit cells → Cluster & edit) auf `person_label` (Method: key collision + metaphone3). Prüfe Zusammenführungen.
8. Export als `architects_roles.csv` (Spalten: `building_row_index` / `person_id` / `person_label`). Optional: füge Quellspalte `ARCHITECTS` hinzu.

### 5. Vorbereitung für Wikidata-Reconciliation

Ziel: Spalte (z. B. `appellation` oder aufbereitete Personennamen) für spätere Abgleichung.

Schritte:

1. Entferne offensichtliche Zusätze (Regex Replace): Beispiel: `value.replace(/,\s*Deutschland$/,'')` (nur wenn sinnvoll!).
2. Normalisiere Großschreibung: `value.toTitlecase()` (sparsam einsetzen, um historische Schreibweisen nicht zu verfälschen).
3. Exportiere Liste eindeutiger Werte (`Facet → Export → tabular`) als `appellations_unique.csv`.

### 6. Audit & Undo/Redo

- Nutze das Undo/Redo Panel, dokumentiere die angewandten Schritte (Screenshot / JSON). Export: „Extract…“ → Speichere Transformations-JSON als `openrefine-history.json` für Reproduzierbarkeit.

### 7. Kurzer Qualitätsbericht (Markdown außerhalb OpenRefine)

- Anzahl ursprünglicher vs. bereinigter `verbaleDating` Tokens.
- Anzahl zusammengeführter Personenlabels durch Clustering.
- Wichtige offene Problemfälle (Stichpunkte).
